<a href="https://colab.research.google.com/github/KSustaina/EDXCS50/blob/main/Degrees2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!which python
!/usr/local/bin/python util.py

/usr/local/bin/python


In [13]:
import csv
import sys
import os

from util import Node, StackFrontier, QueueFrontier

# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}


def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def main():
#    if len(sys.argv) > 2:
#        sys.exit("Usage: python degrees.py [directory]")
#    directory = sys.argv[1] if len(sys.argv) == 2 else "large"
    directory = "large"
    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")

def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.

    If there is no possible path, returns None.
    """

    # Initialize the queue with the starting node and its path.
    queue = [(source, [])]
    # Keep track of visited nodes to avoid cycles.
    visited = {source}
    print(visited)
    while queue:
        # Get the next node and its path from the queue.
        current_node, path = queue.pop(0)

        # If we've reached the target, return the path.
        if current_node == target:
            return path

        # Get the neighbors of the current node.
        neighbors = neighbors_for_person(current_node)

        # Add unvisited neighbors to the queue with their updated paths.
        for neighbor in neighbors:
            next_node = neighbor[1]  # Get the person_id from the neighbor tuple
            if next_node not in visited:
                visited.add(next_node)
                queue.append((next_node, path + [neighbor]))  # Add neighbor to path

    # If no path is found, return None.
    return None

    # raise NotImplementedError


def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors



if __name__ == "__main__":
    main()

Loading data...
Data loaded.
Name: john wayne
Name: marlon brando
{'78'}
3 degrees of separation.
1: John Wayne and Ann-Margret starred in The Train Robbers
2: Ann-Margret and Robert Duvall starred in Newsies
3: Robert Duvall and Marlon Brando starred in Apocalypse Now
